# Grad-CAM

- [jacobgil/pytorch-grad-cam](https://github.com/jacobgil/pytorch-grad-cam)を使用したEfficientNet-b4の可視化.
- EfficientNetのモデルは[lukemelas/EfficientNet-PyTorch](https://github.com/lukemelas/EfficientNet-PyTorch)を使用. [timm](https://github.com/huggingface/pytorch-image-models)のEfficientNetを使用した場合は画像の右上にheatmapが偏って表示されてしまう（原因は不明）.

In [104]:
import timm
import torch
import pathlib
import torchvision
from PIL import Image
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from efficientnet_pytorch import EfficientNet

In [110]:
image_path = pathlib.Path("../data/63000.png")
image = Image.open(str(image_path))

input_tensor = torchvision.transforms.functional.to_tensor(image)
input_tensor = torchvision.transforms.functional.resize(input_tensor, (380, 380))

normalized_input_tensor = torchvision.transforms.functional.normalize(input_tensor, mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])

In [106]:
model = EfficientNet.from_pretrained("efficientnet-b4")
model.eval()

Loaded pretrained weights for efficientnet-b4


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        48, 12, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        12, 48, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False
  

In [107]:
target_layers = [model._conv_head]

In [108]:
cam = GradCAM(model=model, target_layers=target_layers)
grayscale_cam = cam(input_tensor=normalized_input_tensor[None,:,:,:], targets=None)
grayscale_cam = grayscale_cam[0, :]

In [111]:
# (H, W, C)の順に変換
cam_image = show_cam_on_image(input_tensor.permute(1, 2, 0).cpu().data.numpy(), grayscale_cam, use_rgb=True)

output_path = pathlib.Path("../outputs/gradcam.png")
Image.fromarray(cam_image).save(str(output_path))